동기처리 : 직렬(순차적)로 작업을 수행하는 방식

전체 실행 시간이 느림

비동기 처리 : 병렬(동시)로 작업을 수행하는 방식

In [8]:
# 딜레이 제공 
import time

# 날짜 처리
import datetime as dt

# 메일발송(처리속도차이확인)
import MyMailer

# 비동기 처리 기능 제공
import concurrent.futures as futures



In [9]:
def timeWork(name,seconds):
    print(f'[{name}] 작업을 {seconds}초 동안 수행합니다.')
    for i in range(seconds):
        time.sleep(1)
        print(f'[{name}] {i+1}초...')

    print(f"[{name}] 작업이 종료되었습니다.")

In [6]:
startTime = dt.datetime.now()


timeWork('B', 5)
timeWork('C', 2)

endTime = dt.datetime.now()
workTime = endTime - startTime
print(f'작업에 소요된 시간은 총 {workTime.seconds}초 입니다.')

[A] 작업을 3초 동안 수행합니다.
[A] 1초...
[A] 2초...
[A] 3초...
[A] 작업이 종료되었습니다.
[B] 작업을 5초 동안 수행합니다.
[B] 1초...
[B] 2초...
[B] 3초...
[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
[C] 작업을 2초 동안 수행합니다.
[C] 1초...
[C] 2초...
[C] 작업이 종료되었습니다.
작업에 소요된 시간은 총 10초 입니다.


In [11]:
startTime = dt.datetime.now()

with futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.submit(timeWork,'A', 3)
    executor.submit(timeWork,'B', 5)
    executor.submit(timeWork,'C', 2)

endTime = dt.datetime.now()
workTime = endTime - startTime
print(f'작업에 소요된 시간은 총 {workTime.seconds}초 입니다.')

[A] 작업을 3초 동안 수행합니다.
[B] 작업을 5초 동안 수행합니다.
[C] 작업을 2초 동안 수행합니다.
[A] 1초...[B] 1초...

[C] 1초...
[A] 2초...[B] 2초...

[C] 2초...
[C] 작업이 종료되었습니다.
[B] 3초...[A] 3초...
[A] 작업이 종료되었습니다.

[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
작업에 소요된 시간은 총 5초 입니다.


In [12]:
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = '운영지원팀 <hyk2202@gmail.com>'

subjectTmpl = '{name}님의 {yy}년 {mm}월 급여명세서 입니다.'

startTime = dt.datetime.now()

with open('mail/content.txt','r',encoding='utf-8') as f:
    contentTmpl = f.read()

with open('mail/mail_list.csv','r',encoding='euc-kr') as f:
    csv = f.readlines()
    # print(csv)

    for line in csv:
        name,email,file1,file2 = line.strip().split(',')
        # print(name)

        toAddr = '{name} <{email}>'.format(name=name,email=email)
        # print(toAddr)

        subject = subjectTmpl.format(name =name, yy = year, mm = month)
        # print(subject)

        content = contentTmpl.format(name = name, yy = year, mm = month, dd= day)
        # print(content)

        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print(f'작업에 소요된 시간은 총 {workTime.seconds}초 입니다.')
    

작업에 소요된 시간은 총 34초 입니다.


In [13]:
startTime = dt.datetime.now()

with open('mail/content.txt','r',encoding='utf-8') as f:
    contentTmpl = f.read()

with open('mail/mail_list.csv','r',encoding='euc-kr') as f:
    csv = f.readlines()
    # print(csv)

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name,email,file1,file2 = line.strip().split(',')
            # print(name)

            toAddr = '{name} <{email}>'.format(name=name,email=email)
            # print(toAddr)

            subject = subjectTmpl.format(name =name, yy = year, mm = month)
            # print(subject)

            content = contentTmpl.format(name = name, yy = year, mm = month, dd= day)
            # print(content)

            # MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print(f'작업에 소요된 시간은 총 {workTime.seconds}초 입니다.')
    

작업에 소요된 시간은 총 3초 입니다.
